# Thử nghiệm nâng cao: Ensemble, Multivariate & Hybrid Models

## Phạm vi thử nghiệm
1. **Ensemble đơn biến**: Kết hợp Linear + DLinear + NLinear
2. **Đa biến (Multivariate)**: Sử dụng OHLCV features
3. **Hybrid DLinear + Transformer**: Decomposition + Attention

## 1. Setup & Import

In [ ]:
import os
import random
import warnings
import math
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

## 2. Load & Preprocess Data

In [ ]:
df = pd.read_csv('data/FPT_train.csv')
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values('time').reset_index(drop=True)

print(f"Shape: {df.shape}")
print(f"Date range: {df['time'].min()} → {df['time'].max()}")
display(df.head())

In [ ]:
# Feature engineering
df['close_log'] = np.log(df['close'])
df['open_log'] = np.log(df['open'])
df['high_log'] = np.log(df['high'])
df['low_log'] = np.log(df['low'])
df['volume_log'] = np.log(df['volume'] + 1)
df['hl_spread'] = (df['high'] - df['low']) / df['close']
df['oc_spread'] = (df['close'] - df['open']) / df['open']
df = df.ffill().bfill()

# Split ratios
train_ratio = 0.7
val_ratio = 0.15
n_total = len(df)
train_cutoff = int(n_total * train_ratio)

print(f"Train cutoff: {train_cutoff}")

In [ ]:
# Scaling
# Univariate
uni_scaler = StandardScaler()
uni_scaler.fit(df['close_log'].values[:train_cutoff].reshape(-1, 1))
df['close_scaled'] = uni_scaler.transform(df['close_log'].values.reshape(-1, 1)).flatten()

# Multivariate - scale từng feature riêng
multi_cols = ['open_log', 'high_log', 'low_log', 'close_log', 'volume_log', 'hl_spread', 'oc_spread']
multi_scaler = StandardScaler()
multi_scaler.fit(df[multi_cols].values[:train_cutoff])
multi_scaled = multi_scaler.transform(df[multi_cols].values)

print(f"Univariate feature: close_scaled")
print(f"Multivariate features: {len(multi_cols)} columns")

## 3. Datasets

In [ ]:
class UnivariateDataset(Dataset):
    def __init__(self, series, seq_len, pred_len):
        self.series = series.astype(np.float32)
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.length = max(0, len(self.series) - seq_len - pred_len + 1)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        x = self.series[idx: idx + self.seq_len]
        y = self.series[idx + self.seq_len: idx + self.seq_len + self.pred_len]
        return torch.from_numpy(x), torch.from_numpy(y)


class MultivariateDataset(Dataset):
    def __init__(self, features, target, seq_len, pred_len):
        self.features = features.astype(np.float32)
        self.target = target.astype(np.float32)
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.length = max(0, len(self.features) - seq_len - pred_len + 1)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        x = self.features[idx: idx + self.seq_len]
        y = self.target[idx + self.seq_len: idx + self.seq_len + self.pred_len]
        return torch.from_numpy(x), torch.from_numpy(y)

In [ ]:
# Config
seq_lengths = {'7d': 7, '30d': 30, '120d': 120, '480d': 480}
pred_len = 100
batch_size = 32
num_epochs = 500
patience = 80
lr = 1e-3

# Prepare data
series_uni = df['close_scaled'].values
# Cho multivariate, target là cột close_log đã scale (index 3)
target_multi = multi_scaled[:, 3]  # close_log_scaled
n_features = len(multi_cols)
close_idx = 3

# Create datasets
uni_datasets = {}
multi_datasets = {}

for name, seq_len in seq_lengths.items():
    uni_datasets[name] = UnivariateDataset(series_uni, seq_len, pred_len)
    multi_datasets[name] = MultivariateDataset(multi_scaled, target_multi, seq_len, pred_len)
    print(f"{name}: uni={len(uni_datasets[name])}, multi={len(multi_datasets[name])} samples")

In [ ]:
def create_splits(dataset):
    total = len(dataset)
    train_len = int(total * train_ratio)
    val_len = int(total * val_ratio)
    return {
        'train': torch.utils.data.Subset(dataset, list(range(train_len))),
        'val': torch.utils.data.Subset(dataset, list(range(train_len, train_len + val_len))),
        'test': torch.utils.data.Subset(dataset, list(range(train_len + val_len, total)))
    }

def make_loader(subset, batch_size, shuffle=False):
    if subset is None or len(subset) == 0:
        return None
    return DataLoader(subset, batch_size=batch_size, shuffle=shuffle, drop_last=False)

## 4. Model Definitions

In [ ]:
# Base Models
class Linear(nn.Module):
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.fc = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        # x: (B, seq_len)
        return self.fc(x)


class NLinear(nn.Module):
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.fc = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        # x: (B, seq_len)
        last = x[:, -1:]
        return self.fc(x - last) + last


class DLinear(nn.Module):
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.kernel_size = max(3, seq_len // 4)
        self.fc_trend = nn.Linear(seq_len, pred_len)
        self.fc_seasonal = nn.Linear(seq_len, pred_len)
        
    def forward(self, x):
        # x: (B, seq_len)
        # Simple moving average decomposition
        trend = x.unfold(-1, self.kernel_size, 1).mean(-1)  # (B, seq_len - kernel + 1)
        # Pad trend back to seq_len
        pad_left = (self.seq_len - trend.size(-1)) // 2
        pad_right = self.seq_len - trend.size(-1) - pad_left
        trend = F.pad(trend, (pad_left, pad_right), mode='replicate')
        seasonal = x - trend
        return self.fc_trend(trend) + self.fc_seasonal(seasonal)

In [ ]:
# Ensemble Models
class WeightedEnsemble(nn.Module):
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.linear = Linear(seq_len, pred_len)
        self.dlinear = DLinear(seq_len, pred_len)
        self.nlinear = NLinear(seq_len, pred_len)
        self.weights = nn.Parameter(torch.ones(3) / 3)
        
    def forward(self, x):
        p1 = self.linear(x)
        p2 = self.dlinear(x)
        p3 = self.nlinear(x)
        w = F.softmax(self.weights, dim=0)
        return w[0] * p1 + w[1] * p2 + w[2] * p3
    
    def get_weights(self):
        return F.softmax(self.weights, dim=0).detach().cpu().numpy()


class StackingEnsemble(nn.Module):
    def __init__(self, seq_len, pred_len):
        super().__init__()
        self.linear = Linear(seq_len, pred_len)
        self.dlinear = DLinear(seq_len, pred_len)
        self.nlinear = NLinear(seq_len, pred_len)
        self.meta = nn.Sequential(
            nn.Linear(pred_len * 3, pred_len * 2),
            nn.ReLU(),
            nn.Linear(pred_len * 2, pred_len)
        )
        
    def forward(self, x):
        p1 = self.linear(x)
        p2 = self.dlinear(x)
        p3 = self.nlinear(x)
        combined = torch.cat([p1, p2, p3], dim=-1)
        return self.meta(combined)

In [ ]:
# Multivariate Models
class MultiLinear(nn.Module):
    def __init__(self, seq_len, pred_len, n_features):
        super().__init__()
        self.fc = nn.Linear(seq_len * n_features, pred_len)
        
    def forward(self, x):
        # x: (B, seq_len, n_features)
        B = x.size(0)
        return self.fc(x.reshape(B, -1))


class MultiNLinear(nn.Module):
    def __init__(self, seq_len, pred_len, n_features, close_idx=3):
        super().__init__()
        self.close_idx = close_idx
        self.fc = nn.Linear(seq_len * n_features, pred_len)
        
    def forward(self, x):
        # x: (B, seq_len, n_features)
        B = x.size(0)
        last_close = x[:, -1, self.close_idx:self.close_idx+1]  # (B, 1)
        return self.fc(x.reshape(B, -1)) + last_close


class MultiDLinear(nn.Module):
    def __init__(self, seq_len, pred_len, n_features, close_idx=3):
        super().__init__()
        self.seq_len = seq_len
        self.close_idx = close_idx
        self.kernel_size = max(3, seq_len // 4)
        self.fc_trend = nn.Linear(seq_len * n_features, pred_len)
        self.fc_seasonal = nn.Linear(seq_len * n_features, pred_len)
        
    def forward(self, x):
        # x: (B, seq_len, n_features)
        B = x.size(0)
        # Decompose close feature
        close_seq = x[:, :, self.close_idx]  # (B, seq_len)
        trend = close_seq.unfold(-1, self.kernel_size, 1).mean(-1)
        pad_left = (self.seq_len - trend.size(-1)) // 2
        pad_right = self.seq_len - trend.size(-1) - pad_left
        trend = F.pad(trend, (pad_left, pad_right), mode='replicate')
        
        # Reconstruct with trend/seasonal close
        seasonal = close_seq - trend
        x_trend = x.clone()
        x_seasonal = x.clone()
        x_trend[:, :, self.close_idx] = trend
        x_seasonal[:, :, self.close_idx] = seasonal
        
        return self.fc_trend(x_trend.reshape(B, -1)) + self.fc_seasonal(x_seasonal.reshape(B, -1))

In [ ]:
# Hybrid DLinear + Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=500):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        if d_model > 1:
            pe[:, 1::2] = torch.cos(position * div_term[:d_model//2])
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


class HybridDLinearTransformer(nn.Module):
    def __init__(self, seq_len, pred_len, d_model=32, nhead=2, num_layers=1):
        super().__init__()
        self.seq_len = seq_len
        self.kernel_size = max(3, seq_len // 4)
        
        # Trend branch
        self.fc_trend = nn.Linear(seq_len, pred_len)
        
        # Seasonal branch with Transformer
        self.input_proj = nn.Linear(1, d_model)
        self.pos_enc = PositionalEncoding(d_model, seq_len)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*4,
            dropout=0.1, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_proj = nn.Linear(d_model * seq_len, pred_len)
        
        self.alpha = nn.Parameter(torch.tensor(0.5))
        
    def forward(self, x):
        # x: (B, seq_len)
        B = x.size(0)
        
        # Decompose
        trend = x.unfold(-1, self.kernel_size, 1).mean(-1)
        pad_left = (self.seq_len - trend.size(-1)) // 2
        pad_right = self.seq_len - trend.size(-1) - pad_left
        trend = F.pad(trend, (pad_left, pad_right), mode='replicate')
        seasonal = x - trend
        
        # Trend prediction
        trend_pred = self.fc_trend(trend)
        
        # Seasonal with Transformer
        seasonal_emb = self.input_proj(seasonal.unsqueeze(-1))  # (B, seq_len, d_model)
        seasonal_emb = self.pos_enc(seasonal_emb)
        seasonal_out = self.transformer(seasonal_emb)  # (B, seq_len, d_model)
        seasonal_pred = self.output_proj(seasonal_out.reshape(B, -1))
        
        # Combine
        w = torch.sigmoid(self.alpha)
        return w * trend_pred + (1 - w) * seasonal_pred
    
    def get_alpha(self):
        return torch.sigmoid(self.alpha).item()


class HybridMulti(nn.Module):
    def __init__(self, seq_len, pred_len, n_features, close_idx=3, d_model=32, nhead=2, num_layers=1):
        super().__init__()
        self.seq_len = seq_len
        self.close_idx = close_idx
        self.kernel_size = max(3, seq_len // 4)
        
        # Trend branch
        self.fc_trend = nn.Linear(seq_len * n_features, pred_len)
        
        # Transformer branch
        self.input_proj = nn.Linear(n_features, d_model)
        self.pos_enc = PositionalEncoding(d_model, seq_len)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=d_model*4,
            dropout=0.1, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers)
        self.output_proj = nn.Linear(d_model * seq_len, pred_len)
        
        self.alpha = nn.Parameter(torch.tensor(0.5))
        
    def forward(self, x):
        # x: (B, seq_len, n_features)
        B = x.size(0)
        
        # Trend from close
        close_seq = x[:, :, self.close_idx]
        trend = close_seq.unfold(-1, self.kernel_size, 1).mean(-1)
        pad_left = (self.seq_len - trend.size(-1)) // 2
        pad_right = self.seq_len - trend.size(-1) - pad_left
        trend = F.pad(trend, (pad_left, pad_right), mode='replicate')
        
        x_trend = x.clone()
        x_trend[:, :, self.close_idx] = trend
        trend_pred = self.fc_trend(x_trend.reshape(B, -1))
        
        # Transformer
        x_emb = self.input_proj(x)
        x_emb = self.pos_enc(x_emb)
        x_out = self.transformer(x_emb)
        trans_pred = self.output_proj(x_out.reshape(B, -1))
        
        w = torch.sigmoid(self.alpha)
        return w * trend_pred + (1 - w) * trans_pred

## 5. Training

In [ ]:
class EarlyStopping:
    def __init__(self, patience=80):
        self.patience = patience
        self.best_val = float('inf')
        self.wait = 0
        self.best_state = None

    def step(self, val_loss, model):
        if val_loss < self.best_val - 1e-5:
            self.best_val = val_loss
            self.wait = 0
            self.best_state = deepcopy(model.state_dict())
        else:
            self.wait += 1
        return self.wait >= self.patience


def train_model(model, train_loader, val_loader, epochs, lr, patience, device):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=30)
    stopper = EarlyStopping(patience)
    model.to(device)
    
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = 0
        for bx, by in train_loader:
            bx, by = bx.to(device), by.to(device)
            optimizer.zero_grad()
            pred = model(bx)
            loss = criterion(pred, by)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for bx, by in val_loader:
                bx, by = bx.to(device), by.to(device)
                val_loss += criterion(model(bx), by).item()
        val_loss /= len(val_loader)
        
        scheduler.step(val_loss)
        if stopper.step(val_loss, model):
            break
    
    if stopper.best_state:
        model.load_state_dict(stopper.best_state)
    return model


def evaluate(model, loader, scaler, pred_len, device):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for bx, by in loader:
            bx = bx.to(device)
            preds.append(model(bx).cpu().numpy())
            trues.append(by.numpy())
    
    preds = np.vstack(preds)
    trues = np.vstack(trues)
    
    # Inverse
    preds_log = scaler.inverse_transform(preds.reshape(-1, 1)).reshape(-1, pred_len)
    trues_log = scaler.inverse_transform(trues.reshape(-1, 1)).reshape(-1, pred_len)
    preds_price = np.exp(preds_log)
    trues_price = np.exp(trues_log)
    
    rmse = np.sqrt(mean_squared_error(trues_price.flatten(), preds_price.flatten()))
    mae = mean_absolute_error(trues_price.flatten(), preds_price.flatten())
    
    return {'rmse': rmse, 'mae': mae}

## 6. Run Experiments

In [ ]:
all_results = []

for horizon, seq_len in seq_lengths.items():
    print(f"\n{'='*50}")
    print(f"HORIZON: {horizon} (seq_len={seq_len})")
    print('='*50)
    
    # Prepare loaders
    uni_splits = create_splits(uni_datasets[horizon])
    uni_train = make_loader(uni_splits['train'], batch_size, shuffle=True)
    uni_val = make_loader(uni_splits['val'], batch_size)
    uni_test = make_loader(uni_splits['test'], batch_size)
    
    multi_splits = create_splits(multi_datasets[horizon])
    multi_train = make_loader(multi_splits['train'], batch_size, shuffle=True)
    multi_val = make_loader(multi_splits['val'], batch_size)
    multi_test = make_loader(multi_splits['test'], batch_size)
    
    # --- BASELINE ---
    print("\n[Baseline] NLinear...", end=' ')
    model = NLinear(seq_len, pred_len)
    model = train_model(model, uni_train, uni_val, num_epochs, lr, patience, device)
    metrics = evaluate(model, uni_test, uni_scaler, pred_len, device)
    print(f"RMSE: {metrics['rmse']:.2f}")
    all_results.append({'Horizon': horizon, 'Type': 'Baseline', 'Model': 'NLinear', **metrics})
    
    # --- ENSEMBLE ---
    print("\n[Ensemble] WeightedEnsemble...", end=' ')
    model = WeightedEnsemble(seq_len, pred_len)
    model = train_model(model, uni_train, uni_val, num_epochs, lr, patience, device)
    metrics = evaluate(model, uni_test, uni_scaler, pred_len, device)
    w = model.get_weights()
    print(f"RMSE: {metrics['rmse']:.2f} (w: L={w[0]:.2f}, D={w[1]:.2f}, N={w[2]:.2f})")
    all_results.append({'Horizon': horizon, 'Type': 'Ensemble', 'Model': 'WeightedEnsemble', **metrics})
    
    print("[Ensemble] StackingEnsemble...", end=' ')
    model = StackingEnsemble(seq_len, pred_len)
    model = train_model(model, uni_train, uni_val, num_epochs, lr, patience, device)
    metrics = evaluate(model, uni_test, uni_scaler, pred_len, device)
    print(f"RMSE: {metrics['rmse']:.2f}")
    all_results.append({'Horizon': horizon, 'Type': 'Ensemble', 'Model': 'StackingEnsemble', **metrics})
    
    # --- MULTIVARIATE ---
    print("\n[Multi] MultiLinear...", end=' ')
    model = MultiLinear(seq_len, pred_len, n_features)
    model = train_model(model, multi_train, multi_val, num_epochs, lr, patience, device)
    metrics = evaluate(model, multi_test, uni_scaler, pred_len, device)
    print(f"RMSE: {metrics['rmse']:.2f}")
    all_results.append({'Horizon': horizon, 'Type': 'Multivariate', 'Model': 'MultiLinear', **metrics})
    
    print("[Multi] MultiNLinear...", end=' ')
    model = MultiNLinear(seq_len, pred_len, n_features, close_idx)
    model = train_model(model, multi_train, multi_val, num_epochs, lr, patience, device)
    metrics = evaluate(model, multi_test, uni_scaler, pred_len, device)
    print(f"RMSE: {metrics['rmse']:.2f}")
    all_results.append({'Horizon': horizon, 'Type': 'Multivariate', 'Model': 'MultiNLinear', **metrics})
    
    print("[Multi] MultiDLinear...", end=' ')
    model = MultiDLinear(seq_len, pred_len, n_features, close_idx)
    model = train_model(model, multi_train, multi_val, num_epochs, lr, patience, device)
    metrics = evaluate(model, multi_test, uni_scaler, pred_len, device)
    print(f"RMSE: {metrics['rmse']:.2f}")
    all_results.append({'Horizon': horizon, 'Type': 'Multivariate', 'Model': 'MultiDLinear', **metrics})
    
    # --- HYBRID ---
    print("\n[Hybrid] DLinear+Transformer (Uni)...", end=' ')
    model = HybridDLinearTransformer(seq_len, pred_len, d_model=32, nhead=2, num_layers=1)
    model = train_model(model, uni_train, uni_val, num_epochs, lr*0.5, patience, device)
    metrics = evaluate(model, uni_test, uni_scaler, pred_len, device)
    print(f"RMSE: {metrics['rmse']:.2f} (alpha: {model.get_alpha():.2f})")
    all_results.append({'Horizon': horizon, 'Type': 'Hybrid', 'Model': 'Hybrid-Uni', **metrics})
    
    print("[Hybrid] DLinear+Transformer (Multi)...", end=' ')
    model = HybridMulti(seq_len, pred_len, n_features, close_idx, d_model=32, nhead=2, num_layers=1)
    model = train_model(model, multi_train, multi_val, num_epochs, lr*0.5, patience, device)
    metrics = evaluate(model, multi_test, uni_scaler, pred_len, device)
    print(f"RMSE: {metrics['rmse']:.2f}")
    all_results.append({'Horizon': horizon, 'Type': 'Hybrid', 'Model': 'Hybrid-Multi', **metrics})

## 7. Results Summary

In [ ]:
results_df = pd.DataFrame(all_results)
print("\n" + "="*60)
print("ALL RESULTS (sorted by RMSE)")
print("="*60)
display(results_df.sort_values('rmse').head(20))

In [ ]:
# Best per horizon
print("\nBest model per horizon:")
for h in seq_lengths.keys():
    best = results_df[results_df['Horizon'] == h].sort_values('rmse').iloc[0]
    print(f"{h}: {best['Model']} ({best['Type']}) - RMSE: {best['rmse']:.2f}")

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

colors = {'Baseline': 'black', 'Ensemble': 'steelblue', 'Multivariate': 'coral', 'Hybrid': 'green'}

for idx, h in enumerate(seq_lengths.keys()):
    ax = axes[idx]
    h_df = results_df[results_df['Horizon'] == h].sort_values('rmse')
    bar_colors = [colors[t] for t in h_df['Type']]
    
    bars = ax.barh(range(len(h_df)), h_df['rmse'], color=bar_colors)
    ax.set_yticks(range(len(h_df)))
    ax.set_yticklabels(h_df['Model'], fontsize=9)
    ax.set_xlabel('RMSE')
    ax.set_title(f'{h} Input')
    ax.invert_yaxis()
    
    for bar, val in zip(bars, h_df['rmse']):
        ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, f'{val:.1f}', va='center', fontsize=8)

from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=c, label=l) for l, c in colors.items()]
fig.legend(handles=legend_elements, loc='upper right')
plt.suptitle('RMSE Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Save results
os.makedirs('results', exist_ok=True)
results_df.to_csv('results/advanced_experiments.csv', index=False)
print("Saved to results/advanced_experiments.csv")

## 8. Generate Submission (100 ngày tiếp theo)

In [ ]:
# Chọn best model từ kết quả
best_row = results_df.sort_values('rmse').iloc[0]
best_horizon = best_row['Horizon']
best_model_name = best_row['Model']
best_type = best_row['Type']
best_seq_len = seq_lengths[best_horizon]

print(f"Best model: {best_model_name} ({best_type})")
print(f"Horizon: {best_horizon} (seq_len={best_seq_len})")
print(f"Test RMSE: {best_row['rmse']:.2f}")

In [ ]:
def forecast_future(model, input_data, scaler, device):
    """Dự báo 100 ngày tiếp theo"""
    model.eval()
    with torch.no_grad():
        x = torch.from_numpy(input_data.astype(np.float32)).unsqueeze(0).to(device)
        pred_scaled = model(x).squeeze(0).cpu().numpy()
    
    # Inverse transform
    pred_log = scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
    pred_price = np.exp(pred_log)
    return pred_price


def create_model(model_name, seq_len, pred_len, n_features=None, close_idx=None):
    """Factory function để tạo model"""
    if model_name == 'NLinear':
        return NLinear(seq_len, pred_len)
    elif model_name == 'WeightedEnsemble':
        return WeightedEnsemble(seq_len, pred_len)
    elif model_name == 'StackingEnsemble':
        return StackingEnsemble(seq_len, pred_len)
    elif model_name == 'MultiLinear':
        return MultiLinear(seq_len, pred_len, n_features)
    elif model_name == 'MultiNLinear':
        return MultiNLinear(seq_len, pred_len, n_features, close_idx)
    elif model_name == 'MultiDLinear':
        return MultiDLinear(seq_len, pred_len, n_features, close_idx)
    elif model_name == 'Hybrid-Uni':
        return HybridDLinearTransformer(seq_len, pred_len)
    elif model_name == 'Hybrid-Multi':
        return HybridMulti(seq_len, pred_len, n_features, close_idx)
    else:
        raise ValueError(f"Unknown model: {model_name}")

In [ ]:
# Retrain best model trên 90% data
print(f"\nRetraining {best_model_name} on 90% data...")

is_multi = best_type in ['Multivariate'] or best_model_name == 'Hybrid-Multi'

if is_multi:
    dataset = multi_datasets[best_horizon]
else:
    dataset = uni_datasets[best_horizon]

# 90% train, 10% val
total = len(dataset)
train_len = int(total * 0.9)
train_subset = torch.utils.data.Subset(dataset, list(range(train_len)))
val_subset = torch.utils.data.Subset(dataset, list(range(train_len, total)))

train_loader = make_loader(train_subset, batch_size, shuffle=True)
val_loader = make_loader(val_subset, batch_size)

# Create and train model
if is_multi:
    model = create_model(best_model_name, best_seq_len, pred_len, n_features, close_idx)
else:
    model = create_model(best_model_name, best_seq_len, pred_len)

model = train_model(model, train_loader, val_loader, num_epochs, lr, patience, device)
print("Retraining done!")

In [ ]:
# Generate forecast
print(f"\nGenerating 100-day forecast...")

if is_multi:
    # Lấy seq_len cuối của multivariate data
    input_data = multi_scaled[-best_seq_len:].copy()
else:
    # Lấy seq_len cuối của univariate data
    input_data = series_uni[-best_seq_len:]

future_prices = forecast_future(model, input_data, uni_scaler, device)

print(f"Forecast range: {future_prices.min():.2f} - {future_prices.max():.2f} VND")
print(f"First 5 days: {future_prices[:5]}")
print(f"Last 5 days: {future_prices[-5:]}")

In [ ]:
# Create submission file
os.makedirs('submissions', exist_ok=True)

submission = pd.DataFrame({
    'id': np.arange(1, pred_len + 1),
    'close': future_prices
})

# Save với tên model
submission_path = f'submissions/submission_{best_model_name.lower().replace("-", "_")}_{best_horizon}.csv'
submission.to_csv(submission_path, index=False)

print(f"\nSubmission saved to: {submission_path}")
print(f"Rows (including header): {len(submission) + 1}")
print("\nPreview:")
display(submission.head(10))
display(submission.tail(5))

In [ ]:
# Plot forecast vs historical
fig, ax = plt.subplots(figsize=(14, 6))

# Historical (last 150 days)
hist_days = 150
hist_dates = df['time'].iloc[-hist_days:]
hist_prices = df['close'].iloc[-hist_days:]

# Future dates
last_date = df['time'].iloc[-1]
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=pred_len, freq='B')

ax.plot(hist_dates, hist_prices, 'b-', linewidth=2, label='Historical')
ax.plot(future_dates, future_prices, 'r--', linewidth=2, label=f'Forecast ({best_model_name})')
ax.axvline(last_date, color='gray', linestyle=':', alpha=0.7, label='Forecast start')

ax.set_xlabel('Date')
ax.set_ylabel('Close Price (VND)')
ax.set_title(f'FPT Stock Price Forecast - {best_model_name} ({best_horizon})')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
# Tạo submission cho tất cả models (optional)
print("\nGenerating submissions for all models...")

all_submissions = []

for _, row in results_df.iterrows():
    model_name = row['Model']
    horizon = row['Horizon']
    model_type = row['Type']
    seq_len = seq_lengths[horizon]
    
    is_multi = model_type in ['Multivariate'] or model_name == 'Hybrid-Multi'
    
    # Get dataset
    if is_multi:
        dataset = multi_datasets[horizon]
    else:
        dataset = uni_datasets[horizon]
    
    # Train/val split
    total = len(dataset)
    train_len = int(total * 0.9)
    train_subset = torch.utils.data.Subset(dataset, list(range(train_len)))
    val_subset = torch.utils.data.Subset(dataset, list(range(train_len, total)))
    train_loader = make_loader(train_subset, batch_size, shuffle=True)
    val_loader = make_loader(val_subset, batch_size)
    
    # Create model
    if is_multi:
        model = create_model(model_name, seq_len, pred_len, n_features, close_idx)
    else:
        model = create_model(model_name, seq_len, pred_len)
    
    # Train
    model = train_model(model, train_loader, val_loader, num_epochs, lr, patience, device)
    
    # Forecast
    if is_multi:
        input_data = multi_scaled[-seq_len:].copy()
    else:
        input_data = series_uni[-seq_len:]
    
    prices = forecast_future(model, input_data, uni_scaler, device)
    
    # Save
    sub = pd.DataFrame({'id': np.arange(1, pred_len + 1), 'close': prices})
    filename = f'submissions/submission_{model_name.lower().replace("-", "_")}_{horizon}.csv'
    sub.to_csv(filename, index=False)
    
    all_submissions.append({
        'Model': model_name,
        'Horizon': horizon,
        'Type': model_type,
        'RMSE': row['rmse'],
        'File': filename
    })
    print(f"  {model_name} ({horizon}): {filename}")

print(f"\nTotal submissions: {len(all_submissions)}")

In [ ]:
# Summary table
sub_df = pd.DataFrame(all_submissions).sort_values('RMSE')
print("\nAll submissions (sorted by Test RMSE):")
display(sub_df)